In [1]:
import pandas as pd

# 데이터 불러오기
df = pd.read_csv("./netflix_customer_churn.csv") # 경로변경

# 데이터 기본 정보 요약
print("🔹 데이터 크기:", df.shape)
print("\n🔹 컬럼명:")
print(df.columns.tolist())

print("\n🔹 상위 5개 행:")
display(df.head())

print("\n🔹 결측치 요약:")
print(df.isnull().sum())

print("\n🔹 데이터 타입 정보:")
print(df.dtypes)


🔹 데이터 크기: (5000, 14)

🔹 컬럼명:
['customer_id', 'age', 'gender', 'subscription_type', 'watch_hours', 'last_login_days', 'region', 'device', 'monthly_fee', 'churned', 'payment_method', 'number_of_profiles', 'avg_watch_time_per_day', 'favorite_genre']

🔹 상위 5개 행:


,customer_id,age,gender,subscription_type,watch_hours,last_login_days,region,device,monthly_fee,churned,payment_method,number_of_profiles,avg_watch_time_per_day,favorite_genre
0,a9b75100-82a8-427a-a208-72f24052884a,51,Other,Basic,14.73,29,Africa,TV,8.99,1,Gift Card,1,0.49,Action
1,49a5dfd9-7e69-4022-a6ad-0a1b9767fb5b,47,Other,Standard,0.70,19,Europe,Mobile,13.99,1,Gift Card,5,0.03,Sci-Fi
2,4d71f6ce-fca9-4ff7-8afa-197ac24de14b,27,Female,Standard,16.32,10,Asia,TV,13.99,0,Crypto,2,1.48,Drama
3,d3c72c38-631b-4f9e-8a0e-de103cad1a7d,53,Other,Premium,4.51,12,Oceania,TV,17.99,1,Crypto,2,0.35,Horror
4,4e265c34-103a-4dbb-9553-76c9aa47e946,56,Other,Standard,1.89,13,Africa,Mobile,13.99,1,Crypto,2,0.13,Action



🔹 결측치 요약:
customer_id               0
age                       0
gender                    0
subscription_type         0
watch_hours               0
last_login_days           0
region                    0
device                    0
monthly_fee               0
churned                   0
payment_method            0
number_of_profiles        0
avg_watch_time_per_day    0
favorite_genre            0
dtype: int64

🔹 데이터 타입 정보:
customer_id                object
age                         int64
gender                     object
subscription_type          object
watch_hours               float64
last_login_days             int64
region                     object
device                     object
monthly_fee               float64
churned                     int64
payment_method             object
number_of_profiles          int64
avg_watch_time_per_day    float64
favorite_genre             object
dtype: object


In [2]:
import plotly.express as px

# 이탈 여부 분포
churn_counts = df['churned'].value_counts().sort_index()
labels = ['유지(0)', '이탈(1)']

# 비율 계산
churn_rate = df['churned'].mean() * 100
print(f"전체 이탈률: {churn_rate:.2f}%")

# 1. 원형 그래프 (plotly)
fig = px.pie(values=churn_counts.values, names=labels, color=labels,
             color_discrete_map={'유지(0)':'#4CAF50','이탈(1)':'#F44336'},
             title='전체 고객 이탈 비율 (Churn Rate)')
fig.show()

# 2. 막대 그래프 (plotly)
fig2 = px.bar(x=labels, y=churn_counts.values, color=labels,
              color_discrete_map={'유지(0)':'#4CAF50','이탈(1)':'#F44336'},
              title='고객 유지 vs 이탈 인원수', labels={'x':'이탈 여부','y':'고객 수'})
fig2.show()


전체 이탈률: 50.30%


이탈률이 50.3%라면, 거의 절반의 고객이 이탈한 상태로 꽤 균형 잡힌 타깃 분포.
머신러닝 학습에도 적합한 데이터 구조.

In [4]:
import pandas as pd
import plotly.express as px

# 새 연령대 구간 정의
bins = [0, 18, 25, 35, 45, 55, 65, 120]
labels = ['0-18', '19-25', '26-35', '36-45', '46-55', '56-65', '65+'] # 사회적 생애주기를 기준으로 라벨링

df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=True)

# 연령대별 평균 이탈률 계산
age_churn = df.groupby('age_group')['churned'].mean().reset_index()
age_churn['churned'] = age_churn['churned'] * 100  # 퍼센트 단위 변환

# 시각화 (plotly)
fig = px.bar(age_churn, x='age_group', y='churned', 
             title='연령대별 평균 이탈률 (%)', 
             labels={'age_group':'연령대', 'churned':'이탈률 (%)'},
             color='churned',
             color_continuous_scale='viridis')
fig.update_yaxes(range=[0, 100])
fig.show()

C:\Users\playdata2\AppData\Local\Temp\ipykernel_8156\825395808.py:11: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



0–18세 구간이 가장 높고, 19–35세는 다소 낮으며
46세 이상부터 다시 약간 증가하는 U자형 패턴처럼 보이네요.
실제로 젊은층의 결제 지속성이 낮고 중장년층의 콘텐츠 피로도 등으로 해석 가능.
나름 균등하게 분포

In [5]:
import plotly.express as px

fig = px.box(df, x='churned', y='watch_hours', 
             color='churned',
             color_discrete_map={0:'#4CAF50', 1:'#F44336'},
             title='이탈 여부에 따른 시청시간 분포',
             labels={'churned':'이탈여부 (0=유지, 1=이탈)', 'watch_hours':'총 시청시간 (watch_hours)'})
fig.update_yaxes(type="log")
fig.show()

왼쪽(초록색, 유지 고객) 의 중앙값이 오른쪽(빨간색, 이탈 고객) 보다 확실히 높음.
시청 시간이 짧을수록 이탈 가능성이 높다로 해석.

In [7]:
import plotly.express as px

# 요금제별 고객 수
sub_counts = df['subscription_type'].value_counts().reset_index()
sub_counts.columns = ['subscription_type', 'count']
fig1 = px.bar(sub_counts, x='subscription_type', y='count',
              title='요금제별 고객 수',
              labels={'subscription_type':'요금제', 'count':'고객 수'},
              color='subscription_type',
              color_discrete_sequence=px.colors.qualitative.Pastel)
fig1.show()

# 요금제별 평균 이탈률
plan_churn = df.groupby('subscription_type')['churned'].mean().reset_index()
plan_churn['churned'] = plan_churn['churned'] * 100

fig2 = px.bar(plan_churn, x='subscription_type', y='churned',
              title='요금제별 평균 이탈률 (%)',
              labels={'subscription_type':'요금제', 'churned':'이탈률 (%)'},
              color='churned',
              color_continuous_scale='viridis')
fig2.update_yaxes(range=[0, 100])
fig2.show()

Basic 요금제 이탈률이 약 60%로 가장 높고,
Standard / Premium 요금제는 각각 45% 안팎으로 낮은 편.

저가 요금제(Basic) 고객은 충성도가 낮고, 쉽게 서비스를 떠나는 경향이 있음.
Premium 요금제 고객은 상대적으로 만족도 높거나, 지속 이용 의사가 더 강한 편이라고 해석 가능.

In [9]:
import plotly.express as px

# 지역별 고객 수
region_counts = df['region'].value_counts().reset_index()
region_counts.columns = ['region', 'count']
fig1 = px.bar(region_counts, x='region', y='count',
              title='지역별 고객 수',
              labels={'region':'지역', 'count':'고객 수'},
              color='region',
              color_discrete_sequence=px.colors.qualitative.Pastel)
fig1.show()

# 지역별 평균 이탈률
region_churn = df.groupby('region')['churned'].mean().reset_index()
region_churn['churned'] = region_churn['churned'] * 100
region_churn = region_churn.sort_values('churned', ascending=False)

fig2 = px.bar(region_churn, x='region', y='churned',
              title='지역별 평균 이탈률 (%)',
              labels={'region':'지역', 'churned':'이탈률 (%)'},
              color='churned',
              color_continuous_scale='viridis')
fig2.update_yaxes(range=[0, 100])
fig2.show()

South America, Europe, North America, Asia 순으로 많고, Oceania는 상대적으로 적음. 
균등한 샘플 분포.
지역별 이탈률이 모두 약 50% 전후로 유사.
즉, 이탈 요인이 지역별 문화적 요인보다, 요금제·이용행태·시청시간 등 개별적 요인에 더 큰 영향을 받는 의미로 해석 가능.

In [9]:
import plotly.express as px

# 수치형 변수만 선택
numeric_df = df.select_dtypes(include=['int64', 'float64'])

# 상관계수 계산
corr = numeric_df.corr()

# 히트맵 시각화
fig = px.imshow(corr, text_auto=".2f", aspect="auto", labels=dict(color="Correlation"), color_continuous_scale='cividis')
fig.update_layout(title_text='수치형 변수 간 상관관계 히트맵')
fig.show()


이용 빈도와 시청량이 줄고, 로그인 간격이 길어질수록 이탈 가능성이 급격히 증가한다.
이는 이탈 조기 탐지 모델(Churn Prediction Model) 의 주요 feature 로 바로 활용 가능.